# 🖥️ CLI Examples - PTOF Analysis System

Questo notebook contiene tutti gli snippet di codice per utilizzare gli script CLI del sistema di analisi PTOF.

**⚙️ Configurazione**: Tutti i comandi utilizzano il virtual environment `.venv` del progetto.

**📂 Struttura Directory**:
- `ptof_inbox/` → PDF da analizzare (INSERISCI QUI)
- `ptof_processed/` → PDF archiviati automaticamente

## 📑 Indice
1. [🚀 **Workflow Automatico**](#workflow-automatico) ⭐ PRIORITÀ
2. [🔗 Analisi Multi-Agente Batch](#multi-agent-batch)
3. [☁️ Cloud Agent](#cloud-agent)
4. [🔍 Analisi e Revisione](#analysis-review)
5. [🛠️ Utility](#utilities)
6. [🤖 Automazione Background](#background-automation)
7. [📊 Diagnostica](#diagnostics)

---
## 🚀 Workflow Automatico {#workflow-automatico}

### ⭐ Pipeline Completa: Inbox → Processed

**Cosa fa**:
1. 📥 Legge PDF da `ptof_inbox/`
2. 📝 Converte PDF → Markdown
3. 🤖 Analizza con pipeline multi-agente
4. 📦 Sposta PDF in `ptof_processed/batch_TIMESTAMP/`
5. 📊 Aggiorna CSV dashboard

---
## 🚀 Workflow Completo: Validazione + Analisi (One-Click)

Questa è la cella **principale** per processare nuovi PTOF. Esegue l'intera pipeline automaticamente.

### ⚙️ Configurazione

La cella usa configurazioni di default ottimizzate. Modifica queste variabili se necessario:

| Parametro | Posizione | Descrizione | Default |
|-----------|-----------|-------------|---------|
| `use_llm_if_ambiguous` | Step 0 | Usa Ollama per file ambigui | `True` |
| Modello LLM | `agentic_pipeline.py` | Modello per analisi | `gemma3:27b` |
| Modello Validazione | `ptof_validator.py` | Modello per validazione | `qwen3:32b` |

### 📋 Pipeline Completa

| Step | Descrizione | Output |
|------|-------------|--------|
| 0 | **Pre-Validazione** (heuristics → LLM) | File non-PTOF → `ptof_discarded/` |
| 1 | **Conversione PDF → Markdown** | `ptof_md/*.md` |
| 2 | **Analisi Multi-Agente** (Analyst → Reviewer → Refiner) | `analysis_results/*.json` |
| 3 | **Archiviazione PDF** | `ptof_processed/batch_*/` |
| 4 | **Rebuild CSV** | `data/analysis_summary.csv` |

### 📂 Directory

| Input | Output |
|-------|--------|
| `ptof_inbox/*.pdf` | `analysis_results/*.json` |
| | `ptof_md/*.md` |
| | `data/analysis_summary.csv` |

### ⚠️ Note Importanti

- **Tempo**: ~2-5 minuti per PTOF (dipende dalla lunghezza)
- **Ollama**: Deve essere in esecuzione (`ollama serve`)
- **Memoria**: Consigliati 16GB+ RAM per modelli 27B/32B

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 4: WORKFLOW COMPLETO (VALIDAZIONE + ANALISI)
# ══════════════════════════════════════════════════════════════════════
# 🚀 WORKFLOW COMPLETO: VALIDAZIONE + ANALISI (ONE-CLICK)
# Esegui questa cella per processare automaticamente tutti i PDF in ptof_inbox/
import os
import sys
import shutil
import subprocess
import logging
import re
from pathlib import Path
from datetime import datetime
from glob import glob

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Configurazione
BASE_DIR = Path('/Users/danieledragoni/git/LIste')
os.chdir(BASE_DIR)
sys.path.insert(0, str(BASE_DIR))

INBOX_DIR = BASE_DIR / "ptof_inbox"
PROCESSED_DIR = BASE_DIR / "ptof_processed"
MD_DIR = BASE_DIR / "ptof_md"
ANALYSIS_DIR = BASE_DIR / "analysis_results"
CSV_FILE = BASE_DIR / "data" / "analysis_summary.csv"

# Crea directory
for d in [INBOX_DIR, PROCESSED_DIR, MD_DIR, ANALYSIS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("="*70)
print("🚀 WORKFLOW COMPLETO: VALIDAZIONE + ANALISI")
print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

# =====================================================
# STEP 0: PRE-VALIDAZIONE
# =====================================================
print("\n" + "="*70)
print("🔍 STEP 0: Pre-Validazione PTOF (heuristics → LLM)")
print("="*70)

from src.validation.ptof_validator import PTOFValidator, ValidationResult

validator = PTOFValidator()
inbox_pdfs = list(INBOX_DIR.glob("*.pdf"))

print(f"📥 PDF in inbox: {len(inbox_pdfs)}")

if not inbox_pdfs:
    print("⚠️ Nessun PDF da processare!")
    print("💡 Copia i PDF in ptof_inbox/ e riprova")
else:
    # Valida tutti i PDF
    valid_pdfs = []
    discarded_count = 0
    
    for i, pdf_path in enumerate(inbox_pdfs, 1):
        print(f"\n[{i}/{len(inbox_pdfs)}] 📄 {pdf_path.name}")
        
        report = validator.validate(pdf_path, use_llm_if_ambiguous=True)
        
        if report.result == ValidationResult.VALID_PTOF.value:
            print(f"   ✅ VALIDO (confidence: {report.confidence:.2f}, fase: {report.phase})")
            valid_pdfs.append(pdf_path)
        else:
            print(f"   ❌ SCARTATO: {report.reason}")
            validator.discard(pdf_path, report)
            discarded_count += 1
    
    print(f"\n📊 Validazione: {len(valid_pdfs)} validi, {discarded_count} scartati")
    
    if not valid_pdfs:
        print("\n⚠️ Nessun PTOF valido trovato!")
        print("💡 Per recuperare file scartati: usa la cella 'Recovery'")
    else:
        # =====================================================
        # STEP 1: CONVERSIONE PDF → MARKDOWN
        # =====================================================
        print("\n" + "="*70)
        print("📝 STEP 1: Conversione PDF → Markdown")
        print("="*70)
        
        from src.processing.convert_pdfs_to_md import pdf_to_markdown
        
        converted = []
        code_mapping = {}
        
        for pdf_path in valid_pdfs:
            # Estrai codice dal filename
            basename = pdf_path.stem.upper()
            match = re.match(r'^([A-Z]{2}[A-Z]{2}[A-Z0-9]{6})', basename)
            if match:
                school_code = match.group(1)
            else:
                parts = re.split(r'[_\-\s]', basename)
                school_code = parts[0][:10] if parts else basename[:10]
            
            md_output = MD_DIR / f"{school_code}.md"
            
            # Verifica duplicato
            analysis_file = ANALYSIS_DIR / f"{school_code}_analysis.json"
            if analysis_file.exists():
                print(f"⏭️ Già analizzato: {school_code}")
                continue
            
            print(f"🔄 Convertendo: {pdf_path.name} → {school_code}.md")
            
            try:
                if pdf_to_markdown(str(pdf_path), str(md_output)):
                    converted.append(pdf_path)
                    code_mapping[str(pdf_path)] = school_code
                    print(f"   ✅ Convertito!")
                else:
                    print(f"   ❌ Errore conversione")
            except Exception as e:
                print(f"   ❌ Errore: {e}")
        
        print(f"\n📊 Convertiti: {len(converted)} file")
        
        if converted:
            # =====================================================
            # STEP 2: ANALISI MULTI-AGENTE
            # =====================================================
            print("\n" + "="*70)
            print("🤖 STEP 2: Analisi Multi-Agente")
            print("="*70)
            
            from app.agentic_pipeline import (
                AnalystAgent, ReviewerAgent, RefinerAgent, SynthesizerAgent,
                process_single_ptof
            )
            
            # Inizializza agenti
            print("🔧 Inizializzazione agenti...")
            analyst = AnalystAgent()
            reviewer = ReviewerAgent()
            refiner = RefinerAgent()
            synthesizer = SynthesizerAgent()
            print("   ✅ Agenti pronti!")
            
            analyzed = []
            start_time = datetime.now()
            
            for pdf_path in converted:
                school_code = code_mapping.get(str(pdf_path))
                md_file = MD_DIR / f"{school_code}.md"
                
                if not md_file.exists():
                    print(f"⚠️ MD non trovato: {school_code}")
                    continue
                
                print(f"\n🔄 Analizzando: {school_code}")
                
                def status_callback(msg):
                    # Evidenzia log agenti
                    if 'Analyst' in msg or 'Reviewer' in msg or 'Refiner' in msg:
                        print(f"   🤖 {msg}")
                    elif 'chunk' in msg.lower():
                        print(f"   📦 {msg}")
                
                try:
                    result = process_single_ptof(
                        md_file=str(md_file),
                        analyst=analyst,
                        reviewer=reviewer,
                        refiner=refiner,
                        synthesizer=synthesizer,
                        status_callback=status_callback
                    )
                    
                    if result:
                        analyzed.append(school_code)
                        meta = result.get('metadata', {})
                        print(f"   ✅ Completato: {meta.get('denominazione', school_code)[:50]}")
                        
                        # Aggiorna CSV dopo ogni analisi
                        subprocess.run(
                            ['python', 'src/processing/rebuild_csv.py'],
                            capture_output=True, timeout=60
                        )
                    else:
                        print(f"   ⚠️ Nessun risultato")
                        
                except Exception as e:
                    print(f"   ❌ Errore: {e}")
            
            elapsed = (datetime.now() - start_time).total_seconds()
            print(f"\n📊 Analizzati: {len(analyzed)} file in {elapsed:.1f}s")
            
            # =====================================================
            # STEP 3: ARCHIVIAZIONE
            # =====================================================
            print("\n" + "="*70)
            print("📦 STEP 3: Archiviazione PDF")
            print("="*70)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            batch_dir = PROCESSED_DIR / f"batch_{timestamp}"
            batch_dir.mkdir(parents=True, exist_ok=True)
            
            for pdf_path in converted:
                try:
                    dest = batch_dir / pdf_path.name
                    shutil.move(str(pdf_path), str(dest))
                    print(f"   ✅ {pdf_path.name} → processed/")
                except Exception as e:
                    print(f"   ❌ Errore: {e}")
            
            # =====================================================
            # STEP 4: REBUILD CSV FINALE
            # =====================================================
            print("\n" + "="*70)
            print("📊 STEP 4: Aggiornamento CSV Dashboard")
            print("="*70)
            
            result = subprocess.run(
                ['python', 'src/processing/rebuild_csv.py'],
                capture_output=True, text=True, timeout=120
            )
            
            if result.returncode == 0:
                import pandas as pd
                if CSV_FILE.exists():
                    df = pd.read_csv(CSV_FILE)
                    print(f"   ✅ CSV aggiornato: {len(df)} scuole totali")
            else:
                print(f"   ❌ Errore: {result.stderr}")

# =====================================================
# RIEPILOGO FINALE
# =====================================================
print("\n" + "="*70)
print("📊 RIEPILOGO FINALE")
print("="*70)

# Conta file
inbox_count = len(list(INBOX_DIR.glob("*.pdf")))
processed_count = len(list(PROCESSED_DIR.rglob("*.pdf")))
md_count = len(list(MD_DIR.glob("*.md")))
analysis_count = len(list(ANALYSIS_DIR.glob("*.json")))
discarded_list = validator.list_discarded()

print(f"   📥 PDF in inbox: {inbox_count}")
print(f"   ✅ PDF processati: {processed_count}")
print(f"   📝 File Markdown: {md_count}")
print(f"   📊 Analisi JSON: {analysis_count}")
print(f"   🗑️ File scartati: {len(discarded_list)}")

if discarded_list:
    print(f"\n💡 Per recuperare file scartati:")
    print(f"   → Esegui la cella 'Recovery' sotto")

print("\n" + "="*70)
print("✅ WORKFLOW COMPLETATO!")
print("="*70)
print("🔄 Ricarica la dashboard per vedere i nuovi dati")

---
## 🔍 Pre-Validazione PTOF (Solo Validazione)

Questa cella esegue **solo la validazione** dei PDF senza analisi. Usa questa cella per:
- Controllare quali PDF sono veri PTOF prima di analizzarli
- Pulire la cartella inbox da documenti non pertinenti
- Testare il validatore

### ⚙️ Configurazione

| Parametro | Descrizione | Valore Default |
|-----------|-------------|----------------|
| `MOVE_INVALID` | Sposta file non validi in `ptof_discarded/` | `True` |

### 📋 Validazione Progressiva

| Fase | Metodo | Velocità | Quando usato |
|------|--------|----------|--------------|
| 1 | **Heuristics** | ⚡ Veloce | Sempre (keywords, struttura, pagine) |
| 2 | **Ollama LLM** | 🐢 Lento | Solo se ambiguo (confidence 0.2-0.7) |

### 📂 Directory

| Cartella | Contenuto |
|----------|-----------|
| `ptof_inbox/` | PDF da validare |
| `ptof_discarded/not_ptof/` | Circolari, moduli, verbali... |
| `ptof_discarded/too_short/` | Documenti < 5 pagine |
| `ptof_discarded/corrupted/` | PDF illeggibili |

### ⚠️ Nota

Per workflow completo (validazione + analisi), usa la cella "Workflow Completo" sopra.

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 6: PRE-VALIDAZIONE PTOF
# ══════════════════════════════════════════════════════════════════════
# 🔍 PRE-VALIDAZIONE PTOF - Identifica e scarta documenti non-PTOF
# Validazione progressiva: Heuristics (veloce) → Ollama (se ambiguo)
import os
import sys
import logging
from pathlib import Path
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Configurazione
BASE_DIR = Path('/Users/danieledragoni/git/LIste')
os.chdir(BASE_DIR)
sys.path.insert(0, str(BASE_DIR))

PTOF_INBOX_DIR = BASE_DIR / "ptof_inbox"
DISCARDED_DIR = BASE_DIR / "ptof_discarded"

# =====================================================
# INIZIALIZZA VALIDATORE
# =====================================================
from src.validation.ptof_validator import PTOFValidator, ValidationResult

validator = PTOFValidator()

# =====================================================
# 1. MOSTRA STATO ATTUALE
# =====================================================
print("="*70)
print("🔍 PRE-VALIDAZIONE PTOF")
print("="*70)

# Conta PDF
inbox_pdfs = list(PTOF_INBOX_DIR.glob("*.pdf"))
discarded = validator.list_discarded()

print(f"\n📊 STATO ATTUALE:")
print(f"   📥 PDF in inbox: {len(inbox_pdfs)}")
print(f"   🗑️ File scartati: {len(discarded)}")

if inbox_pdfs:
    print(f"\n📄 PDF in inbox:")
    for i, pdf in enumerate(inbox_pdfs[:15], 1):
        print(f"   {i}. {pdf.name}")
    if len(inbox_pdfs) > 15:
        print(f"   ... e altri {len(inbox_pdfs) - 15}")

# =====================================================
# 2. VALIDAZIONE BATCH
# =====================================================
if inbox_pdfs:
    print("\n" + "="*70)
    print("🔍 AVVIO VALIDAZIONE PROGRESSIVA")
    print("   Fase 1: Heuristics (keywords, struttura)")
    print("   Fase 2: Ollama (se ambiguo)")
    print("="*70)
    
    # Imposta True per spostare i file non validi
    MOVE_INVALID = True  # Cambia in False per simulazione
    
    start_time = datetime.now()
    
    results = validator.validate_batch(PTOF_INBOX_DIR, move_invalid=MOVE_INVALID)
    
    elapsed = (datetime.now() - start_time).total_seconds()
    
    print("\n" + "="*70)
    print("📊 RISULTATI VALIDAZIONE")
    print("="*70)
    print(f"   ✅ PTOF validi: {results['stats']['valid']}")
    print(f"   ❌ Non-PTOF: {results['stats']['not_ptof']}")
    print(f"   📄 Troppo corti: {results['stats']['too_short']}")
    print(f"   💔 Corrotti: {results['stats']['corrupted']}")
    print(f"   ❓ Ambigui: {results['stats']['ambiguous']}")
    print(f"   ⏱️ Tempo: {elapsed:.1f}s")
    
    if results['not_ptof'] or results['too_short'] or results['corrupted']:
        print(f"\n📂 File scartati spostati in: ptof_discarded/")
        print("   💡 Per recuperare: usa la sezione 'Recovery' sotto")
else:
    print("\n✅ Nessun PDF da validare in ptof_inbox/")

# =====================================================
# 3. MOSTRA FILE SCARTATI
# =====================================================
discarded = validator.list_discarded()
if discarded:
    print("\n" + "="*70)
    print(f"🗑️ FILE SCARTATI RECUPERABILI ({len(discarded)})")
    print("="*70)
    
    for item in discarded:
        icon = {"not_ptof": "❌", "too_short": "📄", "corrupted": "💔"}.get(item['category'], "❓")
        print(f"   {icon} [{item['category']}] {item['name']}")
        if item.get('report'):
            print(f"      └─ {item['report'].get('reason', 'N/A')}")


print("\n" + "="*70)print("="*70)
print("✅ Validazione completata!")

---
## ♻️ Recovery: Recupera file scartati

Questa cella permette di **recuperare file scartati erroneamente** dalla validazione.

### ⚙️ Configurazione

| Parametro | Descrizione | Valore Default |
|-----------|-------------|----------------|
| `FILENAME_TO_RECOVER` | Nome del file da recuperare | `None` (recupera tutti) |
| `RECOVER_ALL` | Recupera tutti i file scartati | `False` |

### 📋 Cosa fa

1. **Lista file scartati**: Mostra tutti i file in `ptof_discarded/` con motivo dello scarto
2. **Recupero singolo**: Imposta `FILENAME_TO_RECOVER = "nomefile.pdf"` per recuperare un file specifico
3. **Recupero totale**: Imposta `RECOVER_ALL = True` per recuperare tutti i file

### 🚀 Quando usarla

- Dopo aver verificato che un file scartato è effettivamente un PTOF
- Se la validazione LLM ha dato un falso negativo
- Per ri-processare file dopo aver corretto il validatore

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 8: RECOVERY (RECUPERA FILE SCARTATI)
# ══════════════════════════════════════════════════════════════════════
# ♻️ RECOVERY - Recupera file scartati erroneamente
import os
import sys
from pathlib import Path

BASE_DIR = Path('/Users/danieledragoni/git/LIste')
os.chdir(BASE_DIR)
sys.path.insert(0, str(BASE_DIR))

from src.validation.ptof_validator import PTOFValidator

validator = PTOFValidator()

# =====================================================
# 1. LISTA FILE SCARTATI
# =====================================================
discarded = validator.list_discarded()

print("="*70)
print(f"🗑️ FILE SCARTATI RECUPERABILI ({len(discarded)})")
print("="*70)

if not discarded:
    print("\n✅ Nessun file da recuperare")
else:
    for i, item in enumerate(discarded, 1):
        icon = {"not_ptof": "❌", "too_short": "📄", "corrupted": "💔"}.get(item['category'], "❓")
        confidence = item.get('report', {}).get('confidence', 0) if item.get('report') else 0
        reason = item.get('report', {}).get('reason', 'N/A') if item.get('report') else 'N/A'
        
        print(f"\n[{i}] {icon} {item['name']}")
        print(f"    Categoria: {item['category']}")
        print(f"    Motivo: {reason}")
        print(f"    Confidence: {confidence:.2f}")
        print(f"    Path: {item['path']}")

# =====================================================
# 2. RECUPERA UN FILE SPECIFICO
# =====================================================
# ⚠️ MODIFICA il nome del file da recuperare

FILE_DA_RECUPERARE = None  # Es: "MIIS08900V_PTOF.pdf"

if FILE_DA_RECUPERARE:
    print("\n" + "="*70)
    print(f"♻️ RECUPERO: {FILE_DA_RECUPERARE}")
    print("="*70)
    
    # Cerca il file
    for item in discarded:
        if item['name'] == FILE_DA_RECUPERARE:
            result = validator.recover(Path(item['path']))
            if result:
                print(f"✅ Recuperato in: {result}")
            else:
                print(f"❌ Errore recupero")
            break
    else:
        print(f"❌ File non trovato: {FILE_DA_RECUPERARE}")

# =====================================================
# 3. RECUPERA TUTTI I FILE DI UNA CATEGORIA
# =====================================================
# ⚠️ Decommenta una delle righe seguenti per recuperare in batch

# CATEGORIA_DA_RECUPERARE = "not_ptof"   # Documenti non-PTOF
# CATEGORIA_DA_RECUPERARE = "too_short"  # Troppo corti
# CATEGORIA_DA_RECUPERARE = "corrupted"  # Corrotti
CATEGORIA_DA_RECUPERARE = None

if CATEGORIA_DA_RECUPERARE:
    print("\n" + "="*70)
    print(f"♻️ RECUPERO BATCH: categoria '{CATEGORIA_DA_RECUPERARE}'")
    print("="*70)
    
    recovered = validator.recover_all(category=CATEGORIA_DA_RECUPERARE)
    print(f"✅ Recuperati {len(recovered)} file")
    for path in recovered:
        print(f"   → {path.name}")

print("\n" + "="*70)
print("💡 Per recuperare:")
print("   1. Imposta FILE_DA_RECUPERARE = 'nome_file.pdf'")
print("   2. Oppure CATEGORIA_DA_RECUPERARE = 'not_ptof'")
print("="*70)
print("   3. Ri-esegui la cella")

---
## 🔍 Diagnostica CSV ↔ JSON ↔ Database MIUR

Questa cella esegue una **diagnosi completa** delle discrepanze tra:
- File JSON delle analisi (`analysis_results/*.json`)
- CSV della dashboard (`data/analysis_summary.csv`)
- Database MIUR (anagrafica scuole ufficiale)

### ⚙️ Configurazione

| Parametro | Descrizione | Valore Default |
|-----------|-------------|----------------|
| `FIX_JSON` | Corregge automaticamente i JSON con dati MIUR | `True` |

### 📋 Cosa fa

1. **Confronta JSON vs MIUR**: Verifica che comune, regione, provincia nei JSON corrispondano ai dati ufficiali MIUR
2. **Analizza CSV**: Conta righe e mostra anteprima
3. **Correzione automatica**: Se `FIX_JSON=True`, aggiorna i JSON con dati MIUR corretti
4. **Rigenera CSV**: Dopo le correzioni, rigenera il CSV per la dashboard

### 🚀 Quando usarla

- Dopo aver notato discrepanze nella dashboard
- Dopo un batch di analisi per verificare coerenza
- Per correggere dati geografici errati estratti dai PTOF

---
## 🧪 Test Mode: Seleziona PDF per Test

Questa cella permette di testare il workflow su un sottoinsieme di PDF.

### ⚙️ Configurazione

| Parametro | Descrizione | Default |
|-----------|-------------|---------|
| `NUM_TEST_FILES` | Numero di PDF da testare | `10` |
| `MODE` | `"setup"` = prepara test, `"restore"` = ripristina tutti | `"setup"` |

### 📋 Cosa fa

**Modalità `setup`:**
1. Sposta tutti i PDF in `ptof_inbox_backup/`
2. Copia i N PDF più piccoli in `ptof_inbox/`
3. Mostra i file selezionati

**Modalità `restore`:**
1. Ripristina tutti i PDF da backup
2. Elimina la cartella di backup

### 🚀 Workflow consigliato

1. Esegui con `MODE = "setup"` per preparare il test
2. Esegui la cella "Workflow Solo Analisi" (sotto)
3. Verifica i risultati
4. Esegui con `MODE = "restore"` per ripristinare

In [13]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 11: TEST MODE (SELEZIONA PDF PER TEST)
# ══════════════════════════════════════════════════════════════════════
# 🧪 TEST MODE: Seleziona PDF per test rapido
import os
import shutil
from pathlib import Path

# =====================================================
# ⚙️ CONFIGURAZIONE
# =====================================================
MODE = "setup"  # "setup" = prepara test, "restore" = ripristina tutti
NUM_TEST_FILES = 10  # Numero di PDF da testare (seleziona i più piccoli)

# =====================================================
# DIRECTORY
# =====================================================
BASE_DIR = Path('/Users/danieledragoni/git/LIste')
INBOX_DIR = BASE_DIR / "ptof_inbox"
BACKUP_DIR = BASE_DIR / "ptof_inbox_backup"

print("="*70)
print(f"🧪 TEST MODE: {MODE.upper()}")
print("="*70)

if MODE == "setup":
    # =====================================================
    # SETUP: Prepara ambiente di test
    # =====================================================
    
    # Controlla se il backup esiste già
    if BACKUP_DIR.exists() and list(BACKUP_DIR.glob("*.pdf")):
        print("⚠️ Backup già esistente!")
        print(f"   PDF in backup: {len(list(BACKUP_DIR.glob('*.pdf')))}")
        print(f"   PDF in inbox: {len(list(INBOX_DIR.glob('*.pdf')))}")
        print("\n💡 Usa MODE = 'restore' per ripristinare prima di un nuovo test")
    else:
        # Crea backup
        BACKUP_DIR.mkdir(parents=True, exist_ok=True)
        
        # Trova tutti i PDF e ordinali per dimensione
        inbox_pdfs = list(INBOX_DIR.glob("*.pdf"))
        
        if not inbox_pdfs:
            print("⚠️ Nessun PDF in ptof_inbox/")
        else:
            # Ordina per dimensione (più piccoli prima)
            inbox_pdfs_sorted = sorted(inbox_pdfs, key=lambda p: p.stat().st_size)
            
            print(f"📦 Trovati {len(inbox_pdfs)} PDF")
            print(f"📦 Seleziono i {NUM_TEST_FILES} più piccoli per il test")
            
            # Sposta TUTTI in backup
            print(f"\n🔄 Sposto tutti i PDF in backup...")
            for pdf in inbox_pdfs:
                shutil.move(str(pdf), str(BACKUP_DIR / pdf.name))
            print(f"   ✅ {len(inbox_pdfs)} PDF spostati in ptof_inbox_backup/")
            
            # Copia solo i più piccoli in inbox
            test_files = inbox_pdfs_sorted[:NUM_TEST_FILES]
            print(f"\n📋 Copio {len(test_files)} PDF per il test:")
            
            for i, pdf in enumerate(test_files, 1):
                src = BACKUP_DIR / pdf.name
                dst = INBOX_DIR / pdf.name
                shutil.copy(str(src), str(dst))
                size_kb = src.stat().st_size / 1024
                print(f"   {i}. {pdf.name} ({size_kb:.0f} KB)")
            
            print(f"\n✅ Setup completato!")
            print(f"   📥 PDF in inbox (test): {len(list(INBOX_DIR.glob('*.pdf')))}")
            print(f"   📦 PDF in backup: {len(list(BACKUP_DIR.glob('*.pdf')))}")
            print(f"\n🚀 Ora esegui la cella 'Workflow Solo Analisi' sotto")

elif MODE == "restore":
    # =====================================================
    # RESTORE: Ripristina tutti i PDF
    # =====================================================
    
    if not BACKUP_DIR.exists():
        print("⚠️ Nessun backup trovato!")
        print("💡 Non c'è nulla da ripristinare")
    else:
        backup_pdfs = list(BACKUP_DIR.glob("*.pdf"))
        
        if not backup_pdfs:
            print("⚠️ Backup vuoto!")
            # Rimuovi cartella vuota
            BACKUP_DIR.rmdir()
            print("🗑️ Cartella backup rimossa")
        else:
            print(f"🔄 Ripristino {len(backup_pdfs)} PDF dal backup...")
            
            # Rimuovi PDF attuali in inbox (sono copie)
            for pdf in INBOX_DIR.glob("*.pdf"):
                pdf.unlink()
            
            # Sposta tutti dal backup all'inbox
            for pdf in backup_pdfs:
                shutil.move(str(pdf), str(INBOX_DIR / pdf.name))
            
            # Rimuovi cartella backup
            BACKUP_DIR.rmdir()
            
            inbox_count = len(list(INBOX_DIR.glob("*.pdf")))
            print(f"   ✅ {inbox_count} PDF ripristinati in ptof_inbox/")
            print(f"   🗑️ Cartella backup rimossa")
            print(f"\n✅ Restore completato!")

else:
    print(f"❌ MODE non valido: {MODE}")
    print("💡 Usa 'setup' o 'restore'")

print("="*70)


🧪 TEST MODE: SETUP
⚠️ Backup già esistente!
   PDF in backup: 77
   PDF in inbox: 10

💡 Usa MODE = 'restore' per ripristinare prima di un nuovo test


---
## 🚀 Workflow Solo Analisi (Senza Validazione)

Questa cella esegue il workflow **saltando la pre-validazione**. Usa questa cella se:
- I PDF sono già stati validati con la cella "Pre-Validazione"
- Sei sicuro che tutti i PDF in inbox sono PTOF validi
- Vuoi velocizzare il processo saltando il controllo

### ⚙️ Configurazione

Nessuna configurazione necessaria. La cella processa automaticamente tutti i PDF in `ptof_inbox/`.

### 📋 Pipeline

| Step | Descrizione |
|------|-------------|
| 1 | **Conversione PDF → Markdown** (tutti i PDF, nessun filtro) |
| 2 | **Analisi Multi-Agente** (Analyst → Reviewer → Refiner) |
| 3 | **Archiviazione PDF** in `ptof_processed/` |
| 4 | **Rebuild CSV** per la dashboard |

### ⚠️ Differenze dalla cella "Workflow Completo"

| Aspetto | Workflow Completo | Questa Cella |
|---------|-------------------|--------------|
| Pre-validazione | ✅ Sì (heuristics + LLM) | ❌ No |
| Velocità | 🐢 Più lento | ⚡ Più veloce |
| Rischio | Basso (scarta non-PTOF) | Medio (processa tutto) |

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 13: DIAGNOSTICA CSV ↔ JSON ↔ MIUR
# ══════════════════════════════════════════════════════════════════════
# 🔍 DIAGNOSTICA COMPLETA: CSV ↔ JSON ↔ Dashboard ↔ Database MIUR
# Esegui questa cella per capire discrepanze tra le pagine della Dashboard
import os
import sys
import json
import pandas as pd
from pathlib import Path
from glob import glob
import subprocess

BASE_DIR = Path('/Users/danieledragoni/git/LIste')
os.chdir(BASE_DIR)
sys.path.insert(0, str(BASE_DIR))

ANALYSIS_DIR = BASE_DIR / "analysis_results"
CSV_FILE = BASE_DIR / "data" / "analysis_summary.csv"

print("="*70)
print("🔍 DIAGNOSTICA COMPLETA: CSV ↔ JSON ↔ Database MIUR")
print("="*70)

# Carica SchoolDatabase
try:
    from src.utils.school_database import SchoolDatabase
    SCHOOL_DB = SchoolDatabase()
except Exception as e:
    print(f"⚠️ SchoolDatabase non disponibile: {e}")
    SCHOOL_DB = None

# =====================================================
# 1. ANALISI FILE JSON vs DATABASE MIUR
# =====================================================
print("\n📄 1. CONFRONTO JSON vs DATABASE MIUR")
json_files = list(ANALYSIS_DIR.glob("*_analysis.json"))
json_issues = []

for jf in json_files:
    code = jf.stem.replace("_analysis", "").upper()
    try:
        with open(jf, 'r') as f:
            data = json.load(f)
        json_meta = data.get('metadata', {})
        
        # Confronta con database MIUR
        if SCHOOL_DB:
            db_data = SCHOOL_DB.get_school_data(code)
            if db_data:
                # Verifica discrepanze nei campi chiave
                issues = []
                for field in ['comune', 'regione', 'provincia']:
                    json_val = str(json_meta.get(field, '')).upper()
                    db_val = str(db_data.get(field, '')).upper()
                    if json_val and db_val and json_val != db_val and json_val != 'ND':
                        issues.append(f"{field}: JSON='{json_meta.get(field)}' vs MIUR='{db_data.get(field)}'")
                
                if issues:
                    json_issues.append({
                        'code': code,
                        'issues': issues,
                        'db_data': db_data
                    })
                    print(f"   ⚠️ {code}: {', '.join(issues)}")
                else:
                    print(f"   ✅ {code}: dati coerenti con MIUR")
            else:
                print(f"   ❓ {code}: non trovato nel database MIUR")
    except Exception as e:
        print(f"   ❌ {code}: errore lettura JSON - {e}")

# =====================================================
# 2. ANALISI CSV
# =====================================================
print("\n📊 2. CSV analysis_summary.csv")
csv_data = {}
if CSV_FILE.exists():
    df = pd.read_csv(CSV_FILE)
    print(f"   Totale: {len(df)} righe, {len(df.columns)} colonne")
    
    # Mostra anteprima
    cols = ['school_id', 'denominazione', 'regione', 'comune']
    existing = [c for c in cols if c in df.columns]
    print(f"\n   Anteprima:")
    for _, row in df.head(5).iterrows():
        print(f"     {row.get('school_id', 'N/A')}: {row.get('denominazione', 'N/A')[:35]} ({row.get('regione', 'N/A')})")
else:
    print("   ❌ CSV non trovato!")

# =====================================================
# 3. CORREZIONE AUTOMATICA JSON (opzionale)
# =====================================================
if json_issues:
    print("\n" + "="*70)
    print("🔧 CORREZIONE AUTOMATICA DISPONIBILE")
    print("="*70)
    print(f"   Trovate {len(json_issues)} scuole con dati diversi dal database MIUR")
    print("   I JSON verranno corretti con i dati ufficiali del MIUR")
    
    FIX_JSON = True  # Imposta False per simulazione
    
    if FIX_JSON:
        fixed = 0
        for item in json_issues:
            code = item['code']
            db_data = item['db_data']
            json_path = ANALYSIS_DIR / f"{code}_analysis.json"
            
            try:
                with open(json_path, 'r') as f:
                    data = json.load(f)
                
                # Aggiorna metadati con dati MIUR
                if 'metadata' not in data:
                    data['metadata'] = {}
                
                data['metadata']['comune'] = db_data.get('comune', data['metadata'].get('comune', 'ND'))
                data['metadata']['regione'] = db_data.get('regione', data['metadata'].get('regione', 'ND'))
                data['metadata']['provincia'] = db_data.get('provincia', data['metadata'].get('provincia', 'ND'))
                data['metadata']['indirizzo'] = db_data.get('indirizzo', data['metadata'].get('indirizzo', 'ND'))
                data['metadata']['cap'] = db_data.get('cap', data['metadata'].get('cap', 'ND'))
                data['metadata']['email'] = db_data.get('email', data['metadata'].get('email', 'ND'))
                data['metadata']['pec'] = db_data.get('pec', data['metadata'].get('pec', 'ND'))
                data['metadata']['statale_paritaria'] = db_data.get('statale_paritaria', data['metadata'].get('statale_paritaria', 'ND'))
                
                with open(json_path, 'w') as f:
                    json.dump(data, f, ensure_ascii=False, indent=2)
                
                print(f"   ✅ {code}: JSON aggiornato con dati MIUR")
                fixed += 1
            except Exception as e:
                print(f"   ❌ {code}: errore - {e}")
        
        if fixed > 0:
            print(f"\n   📊 Rigenero CSV per riflettere le correzioni...")
            result = subprocess.run(['python', 'src/processing/rebuild_csv.py'], 
                                   capture_output=True, text=True, cwd=str(BASE_DIR))
            if result.returncode == 0:
                print("   ✅ CSV rigenerato!")
            else:
                print(f"   ❌ Errore: {result.stderr}")
else:
    print("\n✅ Tutti i JSON sono coerenti con il database MIUR!")

# =====================================================
# 4. RIEPILOGO FINALE
# =====================================================
print("\n" + "="*70)
print("📋 RIEPILOGO")
print("="*70)
print("✅ FIX APPLICATA: La pagina 'Dati Grezzi' ora usa la regione dal CSV")
print("   (non sovrascrive più con region_map.json)")
print("")
print("Per vedere i dati aggiornati nella dashboard:")
print("   1. Ricarica la dashboard (F5 o Ctrl+R)")
print("="*70)
print("   2. La cache TTL è di 60 secondi")

SyntaxError: invalid syntax (1880549934.py, line 157)

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 14: WORKFLOW COMPLETO ANALISI PTOF
# ══════════════════════════════════════════════════════════════════════
# 🚀 PDF → MD → Analisi Multi-Agente → Arricchimento MIUR → CSV Dashboard
# ✅ Include: validazione codici, arricchimento MIUR, calcolo area_geo/territorio
import sys
import os
import re
import json
import shutil
import subprocess
import logging
from pathlib import Path
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Configurazione
BASE_DIR = Path('/Users/danieledragoni/git/LIste')
os.chdir(BASE_DIR)
sys.path.insert(0, str(BASE_DIR))

INBOX_DIR = BASE_DIR / "ptof_inbox"
PROCESSED_DIR = BASE_DIR / "ptof_processed"
MD_DIR = BASE_DIR / "ptof_md"
ANALYSIS_DIR = BASE_DIR / "analysis_results"
CSV_FILE = BASE_DIR / "data" / "analysis_summary.csv"

# Crea directory
for d in [INBOX_DIR, PROCESSED_DIR, MD_DIR, ANALYSIS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("="*70)
print("🚀 WORKFLOW COMPLETO ANALISI PTOF")
print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

# =====================================================
# COSTANTI PER AREA_GEOGRAFICA E TERRITORIO
# =====================================================
# Funzione per normalizzare regione e ottenere area geografica
def get_area_geografica(regione: str) -> str:
    """Ritorna Nord/Centro/Sud/Isole dalla regione (case-insensitive)"""
    if not regione or regione in ['ND', '', None]:
        return 'ND'
    
    r = regione.upper().strip()
    
    # Nord
    if any(x in r for x in ['PIEMONTE', 'LOMBARDIA', 'VENETO', 'LIGURIA', 
                            'EMILIA', 'FRIULI', 'TRENTINO', 'VALLE']):
        return 'Nord'
    # Centro
    if any(x in r for x in ['TOSCANA', 'UMBRIA', 'MARCHE', 'LAZIO']):
        return 'Centro'
    # Sud
    if any(x in r for x in ['ABRUZZO', 'MOLISE', 'CAMPANIA', 'PUGLIA', 
                            'BASILICATA', 'CALABRIA']):
        return 'Sud'
    # Isole
    if any(x in r for x in ['SICILIA', 'SARDEGNA']):
        return 'Isole'
    
    return 'ND'

PROVINCE_METRO = ['Roma', 'Milano', 'Napoli', 'Torino', 'Bari', 'Firenze', 
                  'Bologna', 'Genova', 'Venezia', 'Palermo', 'Catania', 
                  'Messina', 'Reggio Calabria', 'Cagliari']

# =====================================================
# FUNZIONE: Arricchimento e fix metadati completo
# =====================================================
def complete_metadata_fix(json_path: Path, school_db=None) -> dict:
    """
    Arricchimento completo metadati:
    1. Carica dati MIUR se disponibili
    2. Fallback: deriva provincia/regione dal comune
    3. Calcola area_geografica da regione
    4. Calcola territorio da provincia
    5. Fix nome_scuola da denominazione
    """
    result = {'changes': [], 'warnings': []}
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    if 'metadata' not in data:
        data['metadata'] = {}
    meta = data['metadata']
    changed = False
    
    # Estrai codice dal nome file
    school_code = json_path.stem.replace('_analysis', '')
    if meta.get('school_id') != school_code:
        meta['school_id'] = school_code
        meta['codice_meccanografico'] = school_code
        result['changes'].append(f"school_id = {school_code}")
        changed = True
    
    # FASE 1: Cerca dati MIUR
    if school_db:
        miur_data = school_db.get_school_data(school_code)
        if miur_data:
            miur_fields = {
                'nome_scuola': miur_data.get('denominazione', ''),
                'denominazione': miur_data.get('denominazione', ''),
                'comune': miur_data.get('comune', ''),
                'provincia': miur_data.get('provincia', ''),
                'regione': miur_data.get('regione', ''),
                'tipologia': miur_data.get('tipo_scuola', ''),
            }
            for key, val in miur_fields.items():
                if val and val not in ['ND', '', None]:
                    if meta.get(key) in ['ND', '', None]:
                        meta[key] = val
                        result['changes'].append(f"{key} = '{val}' (MIUR)")
                        changed = True
        else:
            result['warnings'].append(f"Codice {school_code} non in database MIUR")
            
            # FALLBACK: Deriva provincia/regione dal comune
            comune = meta.get('comune', '')
            if comune and comune not in ['ND', '', None] and school_db:
                location = school_db.get_location_by_comune(comune)
                if location:
                    if meta.get('provincia') in ['ND', '', None]:
                        meta['provincia'] = location['provincia']
                        result['changes'].append(f"provincia = '{location['provincia']}' (da comune)")
                        changed = True
                    if meta.get('regione') in ['ND', '', None]:
                        meta['regione'] = location['regione']
                        result['changes'].append(f"regione = '{location['regione']}' (da comune)")
                        changed = True
    
    # FASE 2: Fix nome_scuola da denominazione
    if meta.get('nome_scuola') in ['ND', '', None]:
        denom = meta.get('denominazione', '')
        if denom and denom not in ['ND', '', None]:
            meta['nome_scuola'] = denom
            result['changes'].append(f"nome_scuola = '{denom[:40]}...'")
            changed = True
    
    # FASE 3: Calcola area_geografica da regione (case-insensitive)
    regione = meta.get('regione', '')
    if regione and regione not in ['ND', '', None]:
        area_geo = get_area_geografica(regione)
        if area_geo != 'ND':
            current_area = meta.get('area_geografica', '')
            # Correggi se mancante o errato
            if current_area in ['ND', '', None, 'Metropolitano', 'Non Metropolitano'] or current_area != area_geo:
                meta['area_geografica'] = area_geo
                result['changes'].append(f"area_geografica = '{area_geo}' (da {regione})")
                changed = True
    
    # FASE 4: Calcola territorio da provincia (case-insensitive)
    provincia = meta.get('provincia', '')
    if provincia and provincia not in ['ND', '', None]:
        # Confronto case-insensitive per le province metropolitane
        prov_upper = provincia.upper().strip()
        is_metro = any(p.upper() == prov_upper for p in PROVINCE_METRO)
        territorio = 'Metropolitano' if is_metro else 'Non Metropolitano'
        current_terr = meta.get('territorio', '')
        # Correggi se mancante o errato
        if current_terr in ['ND', '', None, 'Nord', 'Centro', 'Sud', 'Isole'] or current_terr != territorio:
            meta['territorio'] = territorio
            result['changes'].append(f"territorio = '{territorio}' (da {provincia})")
            changed = True
    
    # Salva se modificato
    if changed:
        data['metadata'] = meta
        with open(json_path, 'w') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    
    return result


# =====================================================
# INIZIALIZZAZIONE DATABASE MIUR (con reload forzato)
# =====================================================
print("\n🔧 Caricamento database MIUR...")
import importlib
import src.utils.school_database as school_db_module
importlib.reload(school_db_module)
from src.utils.school_database import SchoolDatabase

# Reset singleton per forzare ricaricamento
SchoolDatabase._instance = None
SchoolDatabase._loaded = False
SCHOOL_DB = SchoolDatabase()
print(f"   ✅ Caricato: {len(SCHOOL_DB._data)} scuole")
print(f"   ✅ Metodo get_location_by_comune: {'presente' if hasattr(SCHOOL_DB, 'get_location_by_comune') else 'MANCANTE!'}")

# Conta PDF
inbox_pdfs = list(INBOX_DIR.glob("*.pdf"))
print(f"\n📥 PDF in inbox: {len(inbox_pdfs)}")

if not inbox_pdfs:
    print("⚠️ Nessun PDF da processare!")
    print("💡 Copia i PDF in ptof_inbox/ e riprova")
else:
    # =====================================================
    # STEP 0: VALIDAZIONE PRE-ANALISI
    # =====================================================
    print("\n" + "="*70)
    print("🔍 STEP 0: Validazione codici meccanografici")
    print("="*70)
    
    valid_pdfs = []
    validation_warnings = []
    
    # Pattern codice: 2 lettere + 2 alfanum + 6 alfanum (es: MIPC09500C)
    code_pattern = re.compile(r'([A-Z]{2}[A-Z0-9]{2}[A-Z0-9]{6})', re.IGNORECASE)
    
    for pdf_path in inbox_pdfs:
        match = code_pattern.search(pdf_path.stem.upper())
        if match:
            school_code = match.group(1).upper()
            miur_data = SCHOOL_DB.get_school_data(school_code)
            
            if miur_data:
                print(f"✅ {school_code}: {miur_data.get('denominazione', 'ND')[:50]}")
                valid_pdfs.append((pdf_path, school_code, miur_data))
            else:
                print(f"⚠️ {school_code}: Non in MIUR (procedo comunque)")
                valid_pdfs.append((pdf_path, school_code, None))
                validation_warnings.append(school_code)
        else:
            print(f"❌ {pdf_path.name}: Codice non estratto")
    
    if validation_warnings:
        print(f"\n⚠️ {len(validation_warnings)} codici non trovati in database MIUR")
    
    print(f"\n📋 PDF da processare: {len(valid_pdfs)}")
    
    # =====================================================
    # STEP 1: CONVERSIONE PDF → MARKDOWN
    # =====================================================
    print("\n" + "="*70)
    print("📝 STEP 1: Conversione PDF → Markdown")
    print("="*70)
    
    from src.processing.convert_pdfs_to_md import pdf_to_markdown
    
    converted = []
    
    for pdf_path, school_code, miur_data in valid_pdfs:
        md_output = MD_DIR / f"{school_code}.md"
        
        # Verifica se già analizzato
        analysis_file = ANALYSIS_DIR / f"{school_code}_analysis.json"
        if analysis_file.exists():
            print(f"⏭️ Già analizzato: {school_code}")
            continue
        
        print(f"🔄 Convertendo: {pdf_path.name} → {school_code}.md")
        
        try:
            if pdf_to_markdown(str(pdf_path), str(md_output)):
                converted.append((pdf_path, school_code, miur_data))
                print(f"   ✅ Convertito!")
            else:
                print(f"   ❌ Errore conversione")
        except Exception as e:
            print(f"   ❌ Errore: {e}")
    
    print(f"\n📊 Convertiti: {len(converted)} file")
    
    if converted:
        # =====================================================
        # STEP 2: ANALISI MULTI-AGENTE
        # =====================================================
        print("\n" + "="*70)
        print("🤖 STEP 2: Analisi Multi-Agente")
        print("="*70)
        
        from app.agentic_pipeline import (
            AnalystAgent, ReviewerAgent, RefinerAgent, SynthesizerAgent,
            process_single_ptof
        )
        
        # Inizializza agenti
        print("🔧 Inizializzazione agenti...")
        analyst = AnalystAgent()
        reviewer = ReviewerAgent()
        refiner = RefinerAgent()
        synthesizer = SynthesizerAgent()
        print("   ✅ Agenti pronti!")
        
        analyzed = []
        start_time = datetime.now()
        
        for pdf_path, school_code, miur_data in converted:
            md_file = MD_DIR / f"{school_code}.md"
            
            if not md_file.exists():
                print(f"⚠️ MD non trovato: {school_code}")
                continue
            
            print(f"\n🔄 Analizzando: {school_code}")
            
            # Info MIUR
            if miur_data:
                print(f"   📍 MIUR: {miur_data.get('denominazione', 'ND')[:40]} - {miur_data.get('comune', 'ND')}")
            
            def status_callback(msg):
                if any(x in msg for x in ['Analyst', 'Reviewer', 'Refiner', 'chunk']):
                    print(f"   🤖 {msg}")
            
            try:
                result = process_single_ptof(
                    md_file=str(md_file),
                    analyst=analyst,
                    reviewer=reviewer,
                    refiner=refiner,
                    synthesizer=synthesizer,
                    status_callback=status_callback
                )
                
                if result:
                    analyzed.append(school_code)
                    print(f"   ✅ Analisi completata")
                    
                    # ARRICCHIMENTO MIUR + FIX METADATI
                    json_path = ANALYSIS_DIR / f"{school_code}_analysis.json"
                    if json_path.exists():
                        fix_result = complete_metadata_fix(json_path, SCHOOL_DB)
                        if fix_result['changes']:
                            print(f"   📝 Arricchito: {len(fix_result['changes'])} campi")
                            for c in fix_result['changes'][:3]:
                                print(f"      • {c}")
                        for w in fix_result['warnings']:
                            print(f"   ⚠️ {w}")
                else:
                    print(f"   ⚠️ Nessun risultato")
                    
            except Exception as e:
                print(f"   ❌ Errore: {e}")
                import traceback
                traceback.print_exc()
        
        elapsed = (datetime.now() - start_time).total_seconds()
        print(f"\n📊 Analizzati: {len(analyzed)} file in {elapsed:.1f}s")
        
        # =====================================================
        # STEP 3: ARCHIVIAZIONE
        # =====================================================
        print("\n" + "="*70)
        print("📦 STEP 3: Archiviazione PDF")
        print("="*70)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        batch_dir = PROCESSED_DIR / f"batch_{timestamp}"
        batch_dir.mkdir(parents=True, exist_ok=True)
        
        for pdf_path, school_code, miur_data in converted:
            try:
                dest = batch_dir / pdf_path.name
                shutil.move(str(pdf_path), str(dest))
                print(f"   ✅ {pdf_path.name} → processed/")
            except Exception as e:
                print(f"   ❌ Errore: {e}")

# =====================================================
# STEP 4: FIX METADATI SU TUTTI I JSON
# =====================================================
print("\n" + "="*70)
print("🔧 STEP 4: Verifica e Fix Metadati (tutti i JSON)")
print("="*70)

json_files = list(ANALYSIS_DIR.glob("*.json"))
fixed_count = 0
for jf in json_files:
    fix_result = complete_metadata_fix(jf, SCHOOL_DB)
    if fix_result['changes']:
        print(f"   🔧 {jf.stem}: {len(fix_result['changes'])} fix")
        for c in fix_result['changes']:
            print(f"      • {c}")
        fixed_count += 1

if fixed_count:
    print(f"\n   ✅ Corretti {fixed_count} file")
else:
    print("   ✅ Tutti i metadati sono già completi")

# =====================================================
# STEP 5: REBUILD CSV DASHBOARD
# =====================================================
print("\n" + "="*70)
print("📊 STEP 5: Rebuild CSV Dashboard")
print("="*70)

result = subprocess.run(
    ['python3', 'src/processing/rebuild_csv.py'],
    capture_output=True, text=True, timeout=120, cwd=str(BASE_DIR)
)

if result.returncode == 0:
    import pandas as pd
    if CSV_FILE.exists():
        df = pd.read_csv(CSV_FILE)
        print(f"   ✅ CSV aggiornato: {len(df)} scuole")
        print(f"\n   Campi principali:")
        for col in ['nome_scuola', 'comune', 'provincia', 'regione', 'area_geografica', 'territorio']:
            non_empty = df[col].notna().sum() if col in df.columns else 0
            print(f"      • {col}: {non_empty}/{len(df)} compilati")
else:
    print(f"   ❌ Errore: {result.stderr}")

# =====================================================
# RIEPILOGO FINALE
# =====================================================
print("\n" + "="*70)
print("📊 RIEPILOGO FINALE")
print("="*70)

inbox_count = len(list(INBOX_DIR.glob("*.pdf")))
processed_count = len(list(PROCESSED_DIR.rglob("*.pdf")))
md_count = len(list(MD_DIR.glob("*.md")))
analysis_count = len(list(ANALYSIS_DIR.glob("*.json")))

print(f"   📥 PDF in inbox: {inbox_count}")
print(f"   ✅ PDF processati: {processed_count}")
print(f"   📝 File Markdown: {md_count}")
print(f"   📊 Analisi JSON: {analysis_count}")

# Verifica metadati completi
print("\n📋 Stato metadati per ogni scuola:")
REQUIRED_FIELDS = ['nome_scuola', 'comune', 'provincia', 'regione', 'area_geografica', 'territorio']
for jf in sorted(ANALYSIS_DIR.glob("*.json")):
    with open(jf, 'r') as f:
        data = json.load(f)
    meta = data.get('metadata', {})
    school_id = meta.get('school_id', jf.stem)
    missing = [k for k in REQUIRED_FIELDS if meta.get(k) in ['ND', '', None]]
    
    if missing:
        print(f"   ⚠️ {school_id}: mancano {missing}")
    else:
        info = f"{meta.get('comune')}, {meta.get('provincia')}"
        geo = f"{meta.get('area_geografica')}/{meta.get('territorio')}"
        print(f"   ✅ {school_id}: {info} ({geo})")

print("\n" + "="*70)
print("✅ WORKFLOW COMPLETATO")
print("="*70)
print("🔄 Ricarica la dashboard per vedere i nuovi dati")

🚀 WORKFLOW COMPLETO ANALISI PTOF
🕐 2025-12-22 17:06:50

🔧 Caricamento database MIUR...


2025-12-22 17:06:50,604 - INFO - [SchoolDatabase] Loaded 61855 schools.


[SchoolDatabase] Loaded 61855 schools from CSVs.
   ✅ Caricato: 61855 scuole
   ✅ Metodo get_location_by_comune: presente

📥 PDF in inbox: 0
⚠️ Nessun PDF da processare!
💡 Copia i PDF in ptof_inbox/ e riprova

🔧 STEP 4: Verifica e Fix Metadati (tutti i JSON)
   🔧 BNIS01100L_analysis: 4 fix
      • school_id = BNIS01100L
      • provincia = 'Lecce' (da comune)
      • regione = 'Puglia' (da comune)
      • nome_scuola = 'Liceo “Virgilio-Redi"...'
   🔧 NAIC80800G_analysis: 3 fix
      • provincia = 'Perugia' (da comune)
      • regione = 'Umbria' (da comune)
      • nome_scuola = 'Istituto Istruzione Superiore Marco Polo...'
   🔧 PCPC00500A_analysis: 3 fix
      • provincia = 'Piacenza' (da comune)
      • regione = 'Emilia Romagna' (da comune)
      • nome_scuola = 'Liceo San Benedetto...'
   🔧 BS1M004009_analysis: 3 fix
      • provincia = 'Brescia' (da comune)
      • regione = 'Lombardia' (da comune)
      • nome_scuola = 'Liceo Paritario “Madonna della Neve”...'

   ✅ Corretti 4 fil

---
## 🔎 Post-Analisi: Scova Incongruenze nei Nomi

Questa cella verifica la coerenza tra i **nomi dei file** e i **codici scuola** nei metadati.

### ⚙️ Configurazione

Nessuna configurazione necessaria.

### 📋 Cosa fa

1. **Scansiona JSON**: Legge tutti i file in `analysis_results/`
2. **Confronta nomi**: Verifica che il nome del file corrisponda a `metadata.school_id`
3. **Segnala problemi**:
   - `rename_json`: Nome file diverso dal codice scuola
   - `conflict_json`: File con nome corretto già esistente
   - `invalid_school_id`: Codice scuola mancante o non valido

### 🚀 Quando usarla

- Dopo un batch di analisi per verificare consistenza
- Se noti problemi con i codici scuola nella dashboard
- Prima di archiviare i risultati finali

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 16: POST-ANALISI INCONGRUENZE NOMI
# ══════════════════════════════════════════════════════════════════════
import os
import json
import glob
from pathlib import Path

BASE_DIR = Path("/Users/danieledragoni/git/LIste")
if not (BASE_DIR / "analysis_results").exists():
    BASE_DIR = Path.cwd()
    if not (BASE_DIR / "analysis_results").exists():
        BASE_DIR = BASE_DIR.parent

ANALYSIS_DIR = str(BASE_DIR / "analysis_results")
MD_DIR = str(BASE_DIR / "ptof_md")



def normalize_school_id(value):
    if value is None:
        return ""
    return str(value).strip()

def is_valid_school_id(value):
    value = normalize_school_id(value)
    if not value or value.upper() in {"ND", "N/A", "NONE", "UNKNOWN"}:
        return False
    return True

def build_issues():
    issues = []
    json_files = glob.glob(os.path.join(ANALYSIS_DIR, "*_analysis.json"))
    for json_path in sorted(json_files):
        filename = os.path.basename(json_path)
        base = filename.replace("_analysis.json", "")
        try:
            with open(json_path, "r") as f:
                data = json.load(f)
        except Exception as e:
            issues.append({
                "kind": "json_read_error",
                "current": json_path,
                "target": "",
                "reason": f"json read error: {e}",
            })
            continue
        school_id = data.get("metadata", {}).get("school_id")
        if not is_valid_school_id(school_id):
            issues.append({
                "kind": "invalid_school_id",
                "current": json_path,
                "target": "",
                "reason": f"metadata.school_id non valido: {school_id}",
            })
            continue
        school_id = normalize_school_id(school_id)
        if base != school_id:
            target_json = os.path.join(ANALYSIS_DIR, f"{school_id}_analysis.json")
            if os.path.exists(target_json):
                issues.append({
                    "kind": "conflict_json",
                    "current": json_path,
                    "target": target_json,
                    "reason": f"dest esiste per {school_id}",
                })
            else:
                issues.append({
                    "kind": "rename_json",
                    "current": json_path,
                    "target": target_json,
                    "reason": f"nome base {base} != metadata.school_id",
                })

            old_analysis_md = os.path.join(ANALYSIS_DIR, f"{base}_analysis.md")
            new_analysis_md = os.path.join(ANALYSIS_DIR, f"{school_id}_analysis.md")
            if os.path.exists(old_analysis_md):
                if os.path.exists(new_analysis_md):
                    issues.append({
                        "kind": "conflict_analysis_md",
                        "current": old_analysis_md,
                        "target": new_analysis_md,
                        "reason": f"dest esiste per {school_id}",
                    })
                else:
                    issues.append({
                        "kind": "rename_analysis_md",
                        "current": old_analysis_md,
                        "target": new_analysis_md,
                        "reason": f"nome base {base} != metadata.school_id",
                    })

            old_md = os.path.join(MD_DIR, f"{base}.md")
            new_md = os.path.join(MD_DIR, f"{school_id}.md")
            if os.path.exists(old_md):
                if os.path.exists(new_md):
                    issues.append({
                        "kind": "conflict_md",
                        "current": old_md,
                        "target": new_md,
                        "reason": f"dest esiste per {school_id}",
                    })
                else:
                    issues.append({
                        "kind": "rename_md",
                        "current": old_md,
                        "target": new_md,
                        "reason": f"nome base {base} != metadata.school_id",
                    })
    return issues

issues = build_issues()
print(f"Totale incongruenze trovate: {len(issues)}")
for idx, issue in enumerate(issues, 1):

    target = issue.get("target") or "-"
    print(f"[{idx:02d}] {issue['kind']} | {os.path.basename(issue['current'])} -> {os.path.basename(target)} | {issue['reason']}")

Totale incongruenze trovate: 3
[01] rename_json | AGPC010001_PTOF_analysis.json -> AGPC010001_analysis.json | nome base AGPC010001_PTOF != metadata.school_id
[02] rename_analysis_md | AGPC010001_PTOF_analysis.md -> AGPC010001_analysis.md | nome base AGPC010001_PTOF != metadata.school_id
[03] rename_md | AGPC010001_PTOF.md -> AGPC010001.md | nome base AGPC010001_PTOF != metadata.school_id


---
## 🔧 Correggi Incongruenze nei Nomi

Questa cella permette di **rinominare i file** per allinearli ai codici scuola corretti.

### ⚙️ Configurazione

| Parametro | Descrizione | Valore Default |
|-----------|-------------|----------------|
| `dry_run` | Simula senza rinominare effettivamente | `True` |
| `selection` | ID da correggere (es. "1,3,5", "all", "none") | Input interattivo |

### 📋 Cosa fa

1. **Mostra piano**: Elenca le rinominazioni proposte
2. **Chiede conferma**: Input interattivo per selezionare quali file rinominare
3. **Esegue rinomina**: Solo se `dry_run = False`

### ⚠️ Note

- Imposta `dry_run = False` nel codice per rinominare effettivamente
- Non sovrascrive file esistenti (skip automatico)
- Dopo la rinomina, rigenera il CSV con la cella "Diagnostica"

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 18: CORREGGI INCONGRUENZE
# ══════════════════════════════════════════════════════════════════════
def safe_rename(src, dst, dry_run=False):
    if os.path.abspath(src) == os.path.abspath(dst):
        return False
    if os.path.exists(dst):
        print(f"⚠️ Skip rename (dest exists): {os.path.basename(dst)}")
        return False
    if dry_run:
        print(f"[DRY] {os.path.basename(src)} -> {os.path.basename(dst)}")
        return True
    os.rename(src, dst)
    print(f"✅ Renamed: {os.path.basename(src)} -> {os.path.basename(dst)}")
    return True

if not issues:
    print("Nessuna incongruenza da correggere.")
else:
    print("Piano: 1) selezione ID  2) rinomina  3) riepilogo")
    selection = input("Quali ID vuoi correggere? (es. 1,3,5 | all | none): ").strip().lower()
    if selection in {"", "none", "no"}:
        print("Nessuna correzione eseguita.")
    else:
        if selection == "all":
            ids = list(range(1, len(issues) + 1))
        else:
            ids = []
            for part in selection.split(','):
                part = part.strip()
                if part.isdigit():
                    ids.append(int(part))
        dry_run = True
        renamed = 0
        for idx in ids:
            if idx < 1 or idx > len(issues):
                continue
            issue = issues[idx - 1]
            if not issue['kind'].startswith('rename_'):
                print(f"- Skip {idx:02d} ({issue['kind']})")
                continue
            if safe_rename(issue['current'], issue['target'], dry_run=dry_run):

                renamed += 1
        print(f"Rinominati: {renamed} (dry_run={dry_run})")

Piano: 1) selezione ID  2) rinomina  3) riepilogo
[DRY] AGPC010001_PTOF_analysis.json -> AGPC010001_analysis.json
[DRY] AGPC010001_PTOF_analysis.md -> AGPC010001_analysis.md
[DRY] AGPC010001_PTOF.md -> AGPC010001.md
Rinominati: 3 (dry_run=True)


---
## 📊 Verifica Stato Directory

Mostra un riepilogo veloce del numero di file in ogni cartella del progetto.

### 📋 Output

| Directory | Descrizione |
|-----------|-------------|
| `ptof_inbox/` | PDF in attesa di processamento |
| `ptof_processed/` | PDF già analizzati |
| `ptof_md/` | File Markdown convertiti |
| `analysis_results/` | Analisi JSON completate |

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 20: VERIFICA STATO DIRECTORY
# ══════════════════════════════════════════════════════════════════════
%%bash
cd /Users/danieledragoni/git/LIste

echo "📊 STATO DIRECTORY"
echo "========================================"
echo "📥 Inbox PDF: $(find ptof_inbox -name '*.pdf' 2>/dev/null | wc -l)"
echo "✅ Processed PDF: $(find ptof_processed -name '*.pdf' 2>/dev/null | wc -l)"

echo "📝 Markdown: $(find ptof_md -name '*.md' 2>/dev/null | wc -l)"echo "========================================"
echo "📊 Analisi JSON: $(find analysis_results -name '*.json' 2>/dev/null | wc -l)"

---
## 🔗 Analisi Multi-Agente Batch (Bash)

Esegue la pipeline multi-agente usando comandi bash. Utile per:
- Debug e testing
- Esecuzione da terminale
- Log più dettagliati

### ⚙️ Configurazione

Modifica il path nel comando se necessario.

### 📋 Pipeline

```
Analyst → Reviewer → Refiner → Synthesizer
```

### ⚠️ Nota

Preferisci le celle Python sopra per un controllo migliore. Questa cella è per uso avanzato.

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 22: PIPELINE MULTI-AGENTE BATCH (BASH)
# ══════════════════════════════════════════════════════════════════════
%%bash
# Pipeline Multi-Agente - Batch completo su cartella ptof_md/
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🚀 Avvio Pipeline Multi-Agente Batch"
echo "📂 Directory: ptof_md/"
echo "🤖 Agenti: Analyst → Reviewer → Refiner"
echo "========================================"

python app/agentic_pipeline.py 2>&1 | tee logs/agentic_pipeline.log


echo "========================================"echo "📋 Log: logs/agentic_pipeline.log"
echo "✅ Pipeline completata!"

---
## 🎯 Analisi Singolo File (con Logging)

Processa un **singolo file PTOF** con logging dettagliato. Utile per debug e test.

### ⚙️ Configurazione

| Parametro | Posizione | Descrizione |
|-----------|-----------|-------------|
| `md_file` | Nel codice | Path al file markdown da analizzare |

### 📋 Cosa fa

1. Inizializza i 4 agenti (Analyst, Reviewer, Refiner, Synthesizer)
2. Processa il singolo file
3. Salva log in `logs/single_analysis.log`

### ⚠️ Nota

Questa cella **non aggiorna** il CSV automaticamente. Usa la cella successiva per l'aggiornamento CSV.

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 24: ANALISI SINGOLO FILE (CON LOGGING)
# ══════════════════════════════════════════════════════════════════════
# Pipeline Multi-Agente - Modalità singolo file con logging dettagliato
import sys
import os
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/single_analysis.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Aggiungi path progetto
sys.path.insert(0, '/Users/danieledragoni/git/LIste')

from app.agentic_pipeline import (
    process_single_ptof,
    AnalystAgent,
    ReviewerAgent,
    RefinerAgent,
    SynthesizerAgent
)

logger.info("🚀 Inizializzazione agenti multi-agente")

# Inizializza agenti
analyst = AnalystAgent()
reviewer = ReviewerAgent()
refiner = RefinerAgent()
synthesizer = SynthesizerAgent()

logger.info("✅ Agenti inizializzati con successo")

# Status callback con logging
def status_callback(msg):
    logger.info(f"[PIPELINE] {msg}")

# Processa un singolo file PTOF
md_file = "ptof_md/MIIS08900V.md"  # Modifica con il tuo file
logger.info(f"📄 Processando file: {md_file}")

start_time = datetime.now()

result = process_single_ptof(
    md_file=md_file,
    analyst=analyst,
    reviewer=reviewer,
    refiner=refiner,
    synthesizer=synthesizer,
    status_callback=status_callback
)

elapsed = (datetime.now() - start_time).total_seconds()

logger.info("="*80)
logger.info(f"✅ Analisi completata in {elapsed:.1f} secondi!")
logger.info(f"📊 Risultato keys: {list(result.keys()) if result else 'None'}")
logger.info(f"📋 Log salvato in: logs/single_analysis.log")

if result:

    metadata = result.get('metadata', {})    logger.info(f"📍 Comune: {metadata.get('comune', 'N/A')}")
    logger.info(f"🏫 Scuola: {metadata.get('denominazione', 'N/A')}")

---
## 🎯 Analisi Singolo File (con CSV Automatico)

Processa un singolo PTOF e **aggiorna automaticamente il CSV** per la dashboard.

### ⚙️ Configurazione

| Parametro | Posizione | Descrizione |
|-----------|-----------|-------------|
| `md_file` | Nel codice | Path al file markdown da analizzare (es. `"ptof_md/MIIS08900V.md"`) |

### 📋 Cosa fa

1. **Verifica percorsi**: Controlla che tutte le directory esistano
2. **Analisi multi-agente**: Analyst → Reviewer → Refiner → Synthesizer
3. **Verifica output**: Controlla che JSON e MD siano stati creati
4. **Aggiorna CSV**: Rigenera `data/analysis_summary.csv`

### 🚀 Quando usarla

- Per analizzare un PTOF specifico
- Per testare la pipeline su un singolo file
- Quando vuoi vedere i risultati subito nella dashboard

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 26: ANALISI SINGOLO FILE (CON CSV AUTOMATICO)
# ══════════════════════════════════════════════════════════════════════
# Pipeline Multi-Agente - Singolo File con AGGIORNAMENTO CSV AUTOMATICO
import sys
import os
import json
import logging
import subprocess
from datetime import datetime
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/single_analysis_csv.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Aggiungi path progetto
BASE_DIR = Path('/Users/danieledragoni/git/LIste')
sys.path.insert(0, str(BASE_DIR))

from app.agentic_pipeline import (
    process_single_ptof,
    AnalystAgent,
    ReviewerAgent,
    RefinerAgent,
    SynthesizerAgent
)

# Directory costanti (relative alla root del progetto)
ANALYSIS_DIR = BASE_DIR / "analysis_results"
MD_DIR = BASE_DIR / "ptof_md"
CSV_FILE = BASE_DIR / "data" / "analysis_summary.csv"
PTOF_PROCESSED_DIR = BASE_DIR / "ptof_processed"
PTOF_INBOX_DIR = BASE_DIR / "ptof_inbox"

def verify_paths_for_dashboard():
    """Verifica che tutti i percorsi siano accessibili dalla dashboard."""
    logger.info("🔍 Verifica percorsi per dashboard...")
    issues = []
    
    # Verifica directory esistono
    for dir_path, name in [
        (ANALYSIS_DIR, "analysis_results"),
        (MD_DIR, "ptof_md"),
        (PTOF_PROCESSED_DIR, "ptof_processed"),
        (PTOF_INBOX_DIR, "ptof_inbox"),
    ]:
        if not dir_path.exists():
            issues.append(f"❌ Directory mancante: {name}")
            dir_path.mkdir(parents=True, exist_ok=True)
            logger.info(f"  📁 Creata directory: {name}")
        else:
            logger.info(f"  ✅ {name}: OK")
    
    # Verifica CSV esiste o può essere creato
    if not CSV_FILE.parent.exists():
        CSV_FILE.parent.mkdir(parents=True, exist_ok=True)
    
    if not CSV_FILE.exists():
        issues.append(f"⚠️ CSV mancante: {CSV_FILE.name} (verrà creato)")
    else:
        logger.info(f"  ✅ CSV: {CSV_FILE.name} OK")
    
    return issues

def update_csv_after_analysis():
    """Aggiorna il CSV dopo l'analisi per la dashboard."""
    logger.info("📊 Aggiornamento CSV per dashboard...")
    try:
        result = subprocess.run(
            ['python', 'src/processing/rebuild_csv.py'],
            cwd=str(BASE_DIR),
            capture_output=True,
            text=True,
            timeout=120
        )
        if result.returncode == 0:
            logger.info("  ✅ CSV aggiornato con successo!")
            # Conta righe
            if CSV_FILE.exists():
                import pandas as pd
                df = pd.read_csv(CSV_FILE)
                logger.info(f"  📈 Totale scuole nel CSV: {len(df)}")
        else:
            logger.error(f"  ❌ Errore rebuild CSV: {result.stderr}")
    except Exception as e:
        logger.error(f"  ❌ Eccezione rebuild CSV: {e}")

def verify_analysis_files(school_code):
    """Verifica che i file dell'analisi siano accessibili."""
    logger.info(f"🔍 Verifica file per {school_code}...")
    
    files_to_check = [
        (ANALYSIS_DIR / f"{school_code}_analysis.json", "JSON analisi"),
        (ANALYSIS_DIR / f"{school_code}_analysis.md", "MD analisi"),
        (MD_DIR / f"{school_code}.md", "MD sorgente"),
    ]
    
    all_ok = True
    for file_path, name in files_to_check:
        if file_path.exists():
            logger.info(f"  ✅ {name}: {file_path.name}")
        else:
            logger.warning(f"  ⚠️ {name}: MANCANTE")
            all_ok = False
    
    # Verifica PDF in processed o inbox
    pdf_found = False
    for pdf_dir in [PTOF_PROCESSED_DIR, PTOF_INBOX_DIR]:
        for pdf in pdf_dir.rglob(f"*{school_code}*.pdf"):
            logger.info(f"  ✅ PDF: {pdf.relative_to(BASE_DIR)}")
            pdf_found = True
            break
        if pdf_found:
            break
    
    if not pdf_found:
        logger.warning(f"  ⚠️ PDF non trovato per {school_code}")
    
    return all_ok

# =====================================================
# ESECUZIONE PRINCIPALE
# =====================================================

logger.info("🚀 Pipeline Multi-Agente con Aggiornamento CSV")
logger.info("="*60)

# 1. Verifica percorsi
path_issues = verify_paths_for_dashboard()
if path_issues:
    for issue in path_issues:
        logger.warning(issue)

# 2. Inizializza agenti
logger.info("\n🤖 Inizializzazione agenti...")
analyst = AnalystAgent()
reviewer = ReviewerAgent()
refiner = RefinerAgent()
synthesizer = SynthesizerAgent()
logger.info("  ✅ Agenti pronti")

# Status callback con logging
def status_callback(msg):
    logger.info(f"  [PIPELINE] {msg}")

# 3. Processa file PTOF
md_file = "ptof_md/ESEMPIO_CODICE.md"  # 👈 MODIFICA CON IL TUO FILE

if not os.path.exists(md_file):
    logger.error(f"❌ File non trovato: {md_file}")
    logger.info("📋 File MD disponibili:")
    for f in sorted(MD_DIR.glob("*.md"))[:10]:
        logger.info(f"    - {f.name}")
else:
    school_code = Path(md_file).stem
    logger.info(f"\n📄 Processando: {md_file}")
    
    start_time = datetime.now()
    
    result = process_single_ptof(
        md_file=md_file,
        analyst=analyst,
        reviewer=reviewer,
        refiner=refiner,
        synthesizer=synthesizer,
        status_callback=status_callback
    )
    
    elapsed = (datetime.now() - start_time).total_seconds()
    
    logger.info("="*60)
    logger.info(f"✅ Analisi completata in {elapsed:.1f} secondi!")
    
    if result:
        metadata = result.get('metadata', {})
        logger.info(f"🏫 Scuola: {metadata.get('denominazione', 'N/A')}")
        logger.info(f"📍 Comune: {metadata.get('comune', 'N/A')}")
        logger.info(f"🗺️ Regione: {metadata.get('regione', 'N/A')}")
        logger.info(f"📮 Provincia: {metadata.get('provincia', 'N/A')}")
    
    # 4. Verifica file creati
    verify_analysis_files(school_code)
    
    # 5. Aggiorna CSV per dashboard
    update_csv_after_analysis()


logger.info("\n" + "="*60)logger.info("📋 Log: logs/single_analysis_csv.log")
logger.info("✅ Pipeline completata!")

---
## 🧠 Verifica Intelligente Percorsi (con LLM)

Usa un LLM per analizzare la coerenza di tutti i file nel progetto.

### ⚙️ Configurazione

| Parametro | Posizione | Descrizione | Default |
|-----------|-----------|-------------|---------|
| `MODEL` | Nel codice | Modello Ollama per analisi | `qwen3:32b` |
| `OLLAMA_URL` | Nel codice | URL server Ollama | `http://192.168.129.14:11434` |

### 📋 Cosa verifica

| Check | Descrizione |
|-------|-------------|
| JSON → MD | Ogni analisi ha il file markdown sorgente |
| JSON → PDF | Ogni analisi ha il PDF corrispondente |
| Metadati | Campi obbligatori presenti (denominazione, regione, ecc.) |
| school_id | Coerenza tra nome file e metadati |

### 🚀 Quando usarla

- Per audit completo del progetto
- Dopo import massivi di nuovi PTOF
- Prima di archiviare risultati finali
- Per diagnosticare problemi nella dashboard

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 28: VERIFICA INTELLIGENTE CON LLM
# ══════════════════════════════════════════════════════════════════════
# Verifica Intelligente Percorsi con LLM
import sys
import os
import json
import logging
import requests
from pathlib import Path
from glob import glob

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

BASE_DIR = Path('/Users/danieledragoni/git/LIste')
sys.path.insert(0, str(BASE_DIR))

ANALYSIS_DIR = BASE_DIR / "analysis_results"
MD_DIR = BASE_DIR / "ptof_md"
PTOF_PROCESSED = BASE_DIR / "ptof_processed"
PTOF_INBOX = BASE_DIR / "ptof_inbox"
CSV_FILE = BASE_DIR / "data" / "analysis_summary.csv"

# Configurazione Ollama
OLLAMA_URL = "http://192.168.129.14:11434/api/generate"
MODEL = "qwen3:32b"  # Modello veloce per verifiche

def call_ollama(prompt, max_tokens=500):
    """Chiama Ollama per analisi intelligente."""
    try:
        response = requests.post(OLLAMA_URL, json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0.1, "num_predict": max_tokens}
        }, timeout=60)
        if response.status_code == 200:
            return response.json().get("response", "")
    except Exception as e:
        logger.warning(f"Ollama non disponibile: {e}")
    return None

def collect_file_inventory():
    """Raccoglie inventario di tutti i file."""
    inventory = {
        'json_files': {},
        'md_files': set(),
        'pdf_files': set(),
        'issues': []
    }
    
    # JSON analisi
    for json_path in ANALYSIS_DIR.glob("*_analysis.json"):
        code = json_path.stem.replace("_analysis", "")
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)
            inventory['json_files'][code] = {
                'path': json_path,
                'metadata': data.get('metadata', {}),
                'has_scores': 'ptof_section2' in data
            }
        except Exception as e:
            inventory['issues'].append(f"JSON corrotto: {json_path.name} - {e}")
    
    # MD sorgente
    for md_path in MD_DIR.glob("*.md"):
        inventory['md_files'].add(md_path.stem)
    
    # PDF (processed e inbox)
    for pdf_dir in [PTOF_PROCESSED, PTOF_INBOX]:
        for pdf in pdf_dir.rglob("*.pdf"):
            # Estrai codice dal nome file
            name = pdf.stem.upper()
            import re
            match = re.search(r'([A-Z]{2,4}[A-Z0-9]{5,8}[A-Z0-9])', name)
            if match:
                inventory['pdf_files'].add(match.group(1))
    
    return inventory

def verify_file_consistency(inventory):
    """Verifica coerenza tra file."""
    issues = list(inventory['issues'])
    
    for code, json_info in inventory['json_files'].items():
        meta = json_info['metadata']
        
        # Verifica MD sorgente esiste
        if code not in inventory['md_files']:
            issues.append(f"⚠️ {code}: MD sorgente mancante in ptof_md/")
        
        # Verifica PDF esiste
        if code.upper() not in inventory['pdf_files']:
            issues.append(f"⚠️ {code}: PDF non trovato in processed/inbox")
        
        # Verifica metadati completi
        required_fields = ['denominazione', 'regione', 'provincia', 'comune']
        missing_meta = [f for f in required_fields if not meta.get(f) or meta.get(f) == 'ND']
        if missing_meta:
            issues.append(f"⚠️ {code}: Metadati mancanti: {', '.join(missing_meta)}")
        
        # Verifica coerenza school_id
        if meta.get('school_id') and meta['school_id'].upper() != code.upper():
            issues.append(f"❌ {code}: school_id nel JSON ({meta['school_id']}) non corrisponde al nome file")
    
    return issues

def llm_analyze_issues(issues, sample_data):
    """Usa LLM per analisi intelligente dei problemi."""
    if not issues:
        return "Nessun problema rilevato. Tutti i file sono coerenti e raggiungibili dalla dashboard."
    
    prompt = f"""/no_think
Sei un esperto di sistemi di analisi PTOF. Analizza questi problemi e suggerisci soluzioni:

PROBLEMI RILEVATI ({len(issues)}):
{chr(10).join(issues[:20])}

CONTESTO:
- Directory analysis_results/: contiene JSON analisi
- Directory ptof_md/: contiene MD sorgenti
- Directory ptof_processed/ e ptof_inbox/: contengono PDF
- Dashboard Streamlit legge da data/analysis_summary.csv

Fornisci:
1. Priorità dei problemi (critico/medio/basso)
2. Comandi o azioni per risolverli
3. Suggerimenti per prevenire problemi futuri

Rispondi in italiano, formato conciso."""

    response = call_ollama(prompt, max_tokens=800)
    return response

# =====================================================
# ESECUZIONE VERIFICA
# =====================================================

logger.info("🔍 VERIFICA INTELLIGENTE PERCORSI FILE")
logger.info("="*60)

# 1. Raccogli inventario
logger.info("\n📂 Raccolta inventario file...")
inventory = collect_file_inventory()
logger.info(f"  JSON analisi: {len(inventory['json_files'])}")
logger.info(f"  MD sorgenti: {len(inventory['md_files'])}")
logger.info(f"  PDF trovati: {len(inventory['pdf_files'])}")

# 2. Verifica coerenza
logger.info("\n🔍 Verifica coerenza file...")
issues = verify_file_consistency(inventory)

if issues:
    logger.warning(f"\n⚠️ PROBLEMI RILEVATI: {len(issues)}")
    for issue in issues[:15]:  # Mostra max 15
        logger.warning(f"  {issue}")
    if len(issues) > 15:
        logger.warning(f"  ... e altri {len(issues) - 15} problemi")
else:
    logger.info("  ✅ Nessun problema rilevato!")

# 3. Analisi LLM (se ci sono problemi)
if issues:
    logger.info("\n🤖 Analisi intelligente con LLM...")
    sample_data = list(inventory['json_files'].items())[:3]
    llm_analysis = llm_analyze_issues(issues, sample_data)
    if llm_analysis:
        logger.info("\n📋 ANALISI LLM:")
        print("-" * 60)
        print(llm_analysis)
        print("-" * 60)
    else:
        logger.info("  LLM non disponibile, verifica manuale consigliata")

# 4. Verifica CSV per dashboard
logger.info("\n📊 Verifica CSV dashboard...")
if CSV_FILE.exists():
    import pandas as pd
    df = pd.read_csv(CSV_FILE)
    logger.info(f"  ✅ CSV esistente: {len(df)} scuole")
    
    # Verifica colonne necessarie
    required_cols = ['school_id', 'denominazione', 'regione', 'provincia', 'email', 'pec']
    missing_cols = [c for c in required_cols if c not in df.columns]
    if missing_cols:
        logger.warning(f"  ⚠️ Colonne mancanti nel CSV: {missing_cols}")
        logger.info("  💡 Esegui: python src/processing/rebuild_csv.py")
else:
    logger.warning("  ❌ CSV non trovato!")
    logger.info("  💡 Esegui: python src/processing/rebuild_csv.py")

logger.info("✅ Verifica completata!")
logger.info("\n" + "="*60)

---
## ☁️ Cloud Agent (Gemini/OpenRouter)

Analisi PTOF usando API cloud invece di Ollama locale. Utile quando:
- Ollama non è disponibile
- Vuoi usare modelli più potenti (GPT-4, Gemini Pro)
- Hai limiti di RAM locale

### ⚙️ Configurazione

| Parametro | Posizione | Descrizione |
|-----------|-----------|-------------|
| `md_file` | Nel codice | File markdown da analizzare |
| API Keys | `data/api_config.json` | Chiavi API per Gemini/OpenRouter |

### 📋 Configurazione API

Crea/modifica `data/api_config.json`:

```json
{
  "default_provider": "gemini",
  "gemini_api_key": "YOUR_GEMINI_API_KEY",
  "openrouter_api_key": "YOUR_OPENROUTER_API_KEY"
}
```

### ⚠️ Note

- Richiede connessione internet
- Potrebbe avere costi (verifica limiti gratuiti)
- Log salvati in `logs/cloud_analysis.log`

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 30: CLOUD AGENT (GEMINI/OPENROUTER)
# ══════════════════════════════════════════════════════════════════════
# Cloud Agent - Analisi completa con logging dettagliato
import sys
import logging
from datetime import datetime

sys.path.insert(0, '/Users/danieledragoni/git/LIste')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/cloud_analysis.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

from src.processing.cloud_review import review_ptof_with_cloud, load_api_config

logger.info("☁️ Inizializzazione Cloud Agent")

# Carica configurazione API dal file JSON
config = load_api_config()
provider = config.get('default_provider', 'gemini')
api_key_field = f'{provider}_api_key'

logger.info(f"🔐 Provider: {provider}")
logger.info(f"🔑 API Key: {'✅ Configurata' if config.get(api_key_field) else '❌ Mancante'}")

# Carica il contenuto del file MD
md_file = 'ptof_md/MIIS08900V.md'
logger.info(f"📄 Caricando file: {md_file}")

with open(md_file, 'r', encoding='utf-8') as f:
    text = f.read()

logger.info(f"📏 Dimensione documento: {len(text):,} caratteri")

# Esegui analisi completa
start_time = datetime.now()
logger.info("🚀 Avvio analisi cloud...")

result = review_ptof_with_cloud(
    md_content=text,
    provider=provider,
    api_key=config.get(api_key_field),
    model="gemini-2.0-flash-exp" if provider == 'gemini' else "google/gemini-2.0-flash-exp:free",
    school_id="MIIS08900V"
)

elapsed = (datetime.now() - start_time).total_seconds()

logger.info("="*80)
logger.info(f"✅ Analisi completata in {elapsed:.1f} secondi!")

if result:
    metadata = result.get('metadata', {})
    logger.info(f"🏫 School ID: {metadata.get('school_id', 'N/A')}")
    logger.info(f"📛 Denominazione: {metadata.get('denominazione', 'N/A')}")
    logger.info(f"📍 Comune: {metadata.get('comune', 'N/A')}")

    logger.info(f"🗺️ Area: {metadata.get('area_geografica', 'N/A')}")logger.info(f"📋 Log salvato in: logs/cloud_analysis.log")
    

---
## 📊 Rebuild CSV

Rigenera il file `data/analysis_summary.csv` leggendo tutti i JSON in `analysis_results/`.

### 🚀 Quando usarla

- Dopo analisi batch per sincronizzare il CSV
- Quando la dashboard mostra dati non aggiornati
- Dopo aver corretto manualmente dei JSON
- Dopo aver usato le celle di correzione incongruenze

### 📋 Output

- `data/analysis_summary.csv`: CSV per la dashboard
- `logs/rebuild_csv.log`: Log dell'operazione

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 32: REBUILD CSV
# ══════════════════════════════════════════════════════════════════════
%%bash
# Rebuild CSV - Ricostruzione indice da file JSON
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "📊 Ricostruzione CSV da file JSON"
echo "📂 Input: analysis_results/*.json"
echo "📄 Output: data/analysis_summary.csv"
echo "========================================"

python src/processing/rebuild_csv.py 2>&1 | tee logs/rebuild_csv.log


echo "========================================"echo "📋 Log: logs/rebuild_csv.log"
echo "✅ CSV ricostruito!"

---
## 🔄 Conversione PDF → Markdown

Converte tutti i PDF in `ptof/` in file Markdown.

### ⚙️ Directory

| Input | Output |
|-------|--------|
| `ptof/*.pdf` | `ptof_md/*.md` |

### 📋 Note

- Usa questa cella solo se hai PDF nella cartella `ptof/`
- Per il workflow normale, i PDF vanno in `ptof_inbox/`
- Log salvato in `logs/pdf_conversion.log`

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 34: CONVERSIONE PDF → MARKDOWN
# ══════════════════════════════════════════════════════════════════════
%%bash
# Conversione PDF → Markdown
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🔄 Conversione PDF → Markdown"
echo "📂 Input: ptof/*.pdf"
echo "📂 Output: ptof_md/*.md"
echo "========================================"

python src/processing/convert_pdfs_to_md.py 2>&1 | tee logs/pdf_conversion.log


echo "========================================"echo "📋 Log: logs/pdf_conversion.log"
echo "✅ Conversione completata!"

---
## 🛠️ Background Fixer

Corregge automaticamente anomalie rilevate nei JSON delle analisi.

### 📋 Cosa fa

1. Legge `data/review_flags.json` per le anomalie segnalate
2. Usa Ollama (qwen2.5-coder:7b) per correggere i dati
3. Aggiorna i JSON in `analysis_results/`

### ⚙️ Prerequisiti

- File `data/review_flags.json` con anomalie
- Ollama in esecuzione con modello qwen2.5-coder:7b

### 📋 Output

- JSON corretti in `analysis_results/`
- Log in `logs/background_fixer.log`

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 36: BACKGROUND FIXER
# ══════════════════════════════════════════════════════════════════════
%%bash
# Background Fixer - Correzione automatica anomalie
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🛠️ Background Fixer - Avvio"
echo "📂 Input: data/review_flags.json"
echo "📂 Output: analysis_results/"
echo "🤖 Modello: qwen2.5-coder:7b (Ollama)"
echo "========================================"

python run_fixer.py 2>&1 | tee logs/background_fixer.log


echo "========================================"echo "📋 Log: logs/background_fixer.log"
echo "✅ Correzione completata!"

---
## 🔍 Diagnostica Sistema

Verifica lo stato completo del sistema di analisi PTOF.

### 📋 Cosa controlla

| Check | Descrizione |
|-------|-------------|
| Anomalie | File con flag di revisione |
| File JSON | Analisi completate |
| File MD | Markdown convertiti |
| PDF inbox | PDF in attesa |
| API Config | Stato chiavi API |

### 📋 Output

- Riepilogo a schermo
- Log dettagliato in `logs/system_diagnostics.log`

In [ ]:
# ══════════════════════════════════════════════════════════════════════
# 📌 CELLA 38: DIAGNOSTICA SISTEMA
# ══════════════════════════════════════════════════════════════════════
# System Status Check - Diagnostica completa con logging
import os
import json
import logging
from glob import glob
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/system_diagnostics.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info("="*80)
logger.info("📊 DIAGNOSTICA SISTEMA PTOF ANALYSIS")
logger.info(f"🕐 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
logger.info("="*80)

# Check review flags
logger.info("\n🚩 ANOMALIE RILEVATE")
if os.path.exists("data/review_flags.json"):
    with open("data/review_flags.json", 'r') as f:
        flags = json.load(f)
    logger.info(f"  File con anomalie: {len(flags)}")
else:
    logger.info("  Nessun file di anomalie trovato")

# Count files
logger.info("\n📄 FILE SISTEMA")
json_files = glob("analysis_results/*.json")
logger.info(f"  File analisi JSON: {len(json_files)}")

md_files = glob("ptof_md/*.md")
logger.info(f"  File Markdown: {len(md_files)}")

pdf_inbox = glob("ptof_inbox/*.pdf")
logger.info(f"  PDF in inbox: {len(pdf_inbox)}")

# API Configuration
logger.info("\n🔐 CONFIGURAZIONE API")
if os.path.exists("data/api_config.json"):
    with open("data/api_config.json", 'r') as f:
        api_config = json.load(f)
    logger.info(f"  Default Provider: {api_config.get('default_provider', 'N/A')}")
    logger.info(f"  Gemini API: {'✅' if api_config.get('gemini_api_key') else '❌'}")
    logger.info(f"  OpenRouter API: {'✅' if api_config.get('openrouter_api_key') else '❌'}")


logger.info("\n="*80)logger.info("📋 Log: logs/system_diagnostics.log")
logger.info("✅ Diagnostica completata!")

---
## 📝 Note

### Best Practices

- ✅ **Virtual Environment**: Tutti i comandi usano `.venv`
- ✅ **Logging**: Ogni operazione salva log in `logs/`
- ✅ **Workflow Automatico**: Usa `workflow_ptof.py` per processare nuovi PDF
- ⚠️ **API Keys**: Configurate in `data/api_config.json`
- ⚠️ **Ollama**: Deve essere in esecuzione per pipeline multi-agente

### Directory

- `ptof_inbox/` - PDF da analizzare
- `ptof_processed/` - PDF archiviati
- `ptof_md/` - Markdown generati
- `analysis_results/` - JSON analisi
- `logs/` - File di log